# Lab Assignment Five: Wide and Deep Network Architectures
In this lab, you will select a prediction task to perform on your dataset, evaluate two different deep learning architectures and tune hyper-parameters for each architecture. If any part of the assignment is not clear, ask the instructor to clarify. 

This report is worth 10% of the final grade. Please upload a report (one per team) with all code used, visualizations, and text in a rendered Jupyter notebook. Any visualizations that cannot be embedded in the notebook, please provide screenshots of the output. The results should be reproducible using your report. Please carefully describe every assumption and every step in your report.

## Dataset Selection

Select a dataset similarly to lab one. That is, the dataset must be table data and must have categorical features. In terms of generalization performance, it is helpful to have a large dataset for building a wide and deep network. It is also helpful to have many different categorical features to create the embeddings and cross-product embeddings. It is fine to perform binary classification, multi-class classification, or regression. You are NOT allowed to use the census (i.e., Adult) dataset that was given as an example in class. 

we have selected this dataset: https://www.kaggle.com/datasets/syedanwarafridi/vehicle-sales-data



## Grading Rubric

### Preparation (4 points total)
- [1 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created). You have the option of using tf.dataset for processing, but it is not required. 

- [1 points] Identify groups of features in your data that should be combined into cross-product features. Provide a compelling justification for why these features should be crossed (or why some features should not be crossed). 

- [1 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

- [1 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Argue why your cross validation method is a realistic mirroring of how an algorithm would be used in practice. Use the method to split your data that you argue for. 


In [132]:
import pandas as pd

df = pd.read_csv('car_prices.csv')
headers = df.columns
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB
None


In [133]:
df.dropna(inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 472325 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          472325 non-null  int64  
 1   make          472325 non-null  object 
 2   model         472325 non-null  object 
 3   trim          472325 non-null  object 
 4   body          472325 non-null  object 
 5   transmission  472325 non-null  object 
 6   vin           472325 non-null  object 
 7   state         472325 non-null  object 
 8   condition     472325 non-null  float64
 9   odometer      472325 non-null  float64
 10  color         472325 non-null  object 
 11  interior      472325 non-null  object 
 12  seller        472325 non-null  object 
 13  mmr           472325 non-null  float64
 14  sellingprice  472325 non-null  float64
 15  saledate      472325 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 61.3+ MB
None


In [134]:
#other years have limited entries, so we will focus on 2005-2014

df = df[df['year'] >= 2005]
df = df[df['year'] < 2015]
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 417707 entries, 2 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          417707 non-null  int64  
 1   make          417707 non-null  object 
 2   model         417707 non-null  object 
 3   trim          417707 non-null  object 
 4   body          417707 non-null  object 
 5   transmission  417707 non-null  object 
 6   vin           417707 non-null  object 
 7   state         417707 non-null  object 
 8   condition     417707 non-null  float64
 9   odometer      417707 non-null  float64
 10  color         417707 non-null  object 
 11  interior      417707 non-null  object 
 12  seller        417707 non-null  object 
 13  mmr           417707 non-null  float64
 14  sellingprice  417707 non-null  float64
 15  saledate      417707 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 54.2+ MB
None


In [135]:
df['year'].value_counts()


year
2013    87467
2012    87380
2014    69712
2011    41384
2008    27011
2007    25378
2010    22616
2006    21631
2009    17959
2005    17169
Name: count, dtype: int64

In [136]:
import pandas as pd

# Define the intervals for the selling price column
intervals = [0,12500, float('inf')]

# Create a new column with the interval labels
df['price_interval'] = pd.cut(df['sellingprice'], bins=intervals)

# Count the number of entries in each interval
interval_counts = df['price_interval'].value_counts()

# Print the number of entries in each interval
print(interval_counts)


price_interval
(12500.0, inf]    219069
(0.0, 12500.0]    198638
Name: count, dtype: int64


In [137]:
unique_trim = df['trim'].unique()
print(unique_trim)

['328i SULEV' '650i' 'Base' ... 'pure' 'EWB' 'Power Wagon']


In [138]:
unique_model = df['model'].unique()
print(unique_model)

['3 Series' '6 Series Gran Coupe' 'M5' 'Cruze' 'A4' 'Camaro' 'A6' 'Q5'
 '6 Series' '5 Series' 'SQ5' 'S5' 'ELR' 'X6' 'ILX' 'A8' 'X1' 'Enclave'
 'TTS' '4 Series' 'MDX' 'Silverado 1500' 'SRX' 'X5' 'G Coupe' 'G Sedan'
 'FX' 'Santa Fe' 'Genesis' 'Equus' 'Elantra' 'Sonata' 'Sonata Hybrid'
 'Accent' 'Veloster' 'Elantra Coupe' 'Azera' 'Tucson' 'Genesis Coupe'
 'Wrangler' 'Optima' 'Altima' 'S-Class' 'GS 350' 'Outlander' 'C-Class'
 'Mazda2' 'Rio' 'M' '370Z' 'Soul' 'Sorento' 'Outlander Sport' 'SLK-Class'
 'ES 350' 'E-Class' 'Mazda3' 'Cooper Clubman' 'Cooper' 'CX-9' 'Forte'
 'Compass' 'JX' 'LR4' 'Mazda5' 'Range Rover Evoque' 'LS 460' 'GLK-Class'
 'Sportage' 'Grand Cherokee' 'MKX' 'XF' 'GL-Class' 'M-Class'
 'Cooper Countryman' 'RX 350' 'Lancer' 'Range Rover Sport' 'Passat'
 'Corolla' 'XC60' 'Sienna' 'Juke' 'Yaris' 'NV' 'CC' 'Leaf' 'Camry'
 'Tacoma' 'Jetta' 'Impreza WRX' 'FJ Cruiser' 'Beetle' 'Avalon' 'FR-S'
 'NV200' 'Rogue' 'Tundra' 'Maxima' 'Cayenne' '911' 'Xterra' 'Versa'
 'Sentra' 'Prius' 'S80' 

In [139]:
unique_makes = df['body'].unique()
print(unique_makes)
len(unique_makes)

['Sedan' 'Convertible' 'SUV' 'Coupe' 'Crew Cab' 'G Coupe' 'G Sedan'
 'Hatchback' 'Elantra Coupe' 'Genesis Coupe' 'Wagon' 'Minivan' 'Van'
 'Double Cab' 'CrewMax Cab' 'Access Cab' 'King Cab' 'CTS Coupe'
 'SuperCrew' 'E-Series Van' 'Extended Cab' 'SuperCab' 'G Convertible'
 'Koup' 'Regular Cab' 'Quad Cab' 'CTS-V Coupe' 'sedan' 'G37 Convertible'
 'Club Cab' 'Q60 Convertible' 'CTS Wagon' 'G37 Coupe' 'Mega Cab'
 'Cab Plus 4' 'Q60 Coupe' 'Beetle Convertible' 'TSX Sport Wagon'
 'Promaster Cargo Van' 'GranTurismo Convertible' 'CTS-V Wagon'
 'convertible' 'minivan' 'van' 'regular-cab' 'suv' 'g sedan' 'g coupe'
 'hatchback' 'king cab' 'supercrew' 'g convertible' 'coupe' 'crew cab'
 'wagon' 'e-series van' 'regular cab' 'quad cab' 'g37 convertible'
 'supercab' 'extended cab' 'crewmax cab' 'double cab' 'genesis coupe'
 'access cab' 'mega cab' 'beetle convertible' 'cts coupe' 'koup'
 'club cab' 'elantra coupe' 'cts-v coupe' 'granturismo convertible'
 'tsx sport wagon' 'promaster cargo van' 'q60 conve

79

In [140]:
df['body'] = df['body'].str.lower()
replacements = {
    'convertible': 'convertible',
    'coupe|koup': 'coupe',
    'cab|crew': 'cab',
    'van': 'van',
    'wagon': 'wagon',
    'sedan': 'sedan'
}

for key, value in replacements.items():
    df.loc[df['body'].str.contains(key, case=False), 'body'] = value

In [141]:
unique_makes = df['body'].unique()
print(unique_makes)
df['body'].value_counts()

['sedan' 'convertible' 'suv' 'coupe' 'cab' 'hatchback' 'wagon' 'van']


body
sedan          195081
suv            105395
cab             35395
van             24024
hatchback       22121
coupe           15177
wagon           12756
convertible      7758
Name: count, dtype: int64

In [142]:
# should we use string for ordinal data? or should we use integers?

ranges = [(0, 11, 'poor'), (11, 21, 'not good'), (21, 31, 'fair'), (31, 41, 'good'), (41, 51, 'great')]

for start, end, label in ranges:
    df['condition'] = df['condition'].replace(range(start, end), label)


In [143]:
from sklearn.model_selection import train_test_split

df_train_orig, df_test_orig = train_test_split(df, test_size=0.2, random_state=37)

In [144]:
from copy import deepcopy
df_train = deepcopy(df_train_orig)
df_test = deepcopy(df_test_orig)

In [145]:
import numpy as np

df_train.reset_index()
df_test.reset_index()

df_test.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,price_interval
387952,2008,Jeep,Liberty,Sport,suv,automatic,1j8gn28k18w195884,oh,great,126044.0,red,gray,tc's used cars llc,6450.0,7800.0,Tue Mar 03 2015 01:30:00 GMT-0800 (PST),"(0.0, 12500.0]"
369170,2014,Lincoln,MKX,Base,suv,automatic,2lmdj8jk4ebl06749,tn,great,10077.0,—,beige,ford motor credit company,36600.0,34000.0,Thu Mar 05 2015 03:00:00 GMT-0800 (PST),"(12500.0, inf]"
165759,2007,GMC,Yukon,SLE,suv,automatic,1gkfk13047r312578,md,great,90057.0,blue,beige,lexus of rockville,16700.0,15400.0,Tue Jan 20 2015 01:30:00 GMT-0800 (PST),"(12500.0, inf]"
113164,2012,Toyota,Prius c,Two,hatchback,automatic,jtdkdtb34c1509187,pa,fair,54808.0,black,gray,ken pollock nissan llc,11900.0,9000.0,Fri Jan 16 2015 01:00:00 GMT-0800 (PST),"(0.0, 12500.0]"
75538,2011,GMC,Yukon,SLT,suv,automatic,1gks1ce05br150916,ca,poor,61831.0,black,black,rvr,23500.0,24500.0,Wed Dec 31 2014 12:30:00 GMT-0800 (PST),"(12500.0, inf]"


In [146]:
#encode the label of target as an integer
from sklearn.preprocessing import LabelEncoder
tmp = LabelEncoder()
df_train.price_interval = tmp.fit_transform(df_train.price_interval)
df_test.price_interval = tmp.transform(df_test.price_interval)

In [147]:
# define variables that should be encoded as integers   
encoders = dict() # save each encoder in dictionary
categorical_headers = ['make','model','body','transmission',
                       'color','interior','state','year','trim','condition']

# get unique values in each feature
for col in categorical_headers:
    print(f'{col} has {len(df_train[col].unique())} unique values.')
    

# ========================================================
# define variables that should be scaled or made discrete
numeric_headers = ['odometer', 'mmr','sellingprice']
df_train[numeric_headers] = df_train[numeric_headers].to_numpy().astype(float)
df_test[numeric_headers] = df_test[numeric_headers].to_numpy().astype(float)
    
df_test.head()

make has 49 unique values.
model has 604 unique values.
body has 8 unique values.
transmission has 2 unique values.
color has 20 unique values.
interior has 17 unique values.
state has 34 unique values.
year has 10 unique values.
trim has 1124 unique values.
condition has 5 unique values.


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,price_interval
387952,2008,Jeep,Liberty,Sport,suv,automatic,1j8gn28k18w195884,oh,great,126044.0,red,gray,tc's used cars llc,6450.0,7800.0,Tue Mar 03 2015 01:30:00 GMT-0800 (PST),0
369170,2014,Lincoln,MKX,Base,suv,automatic,2lmdj8jk4ebl06749,tn,great,10077.0,—,beige,ford motor credit company,36600.0,34000.0,Thu Mar 05 2015 03:00:00 GMT-0800 (PST),1
165759,2007,GMC,Yukon,SLE,suv,automatic,1gkfk13047r312578,md,great,90057.0,blue,beige,lexus of rockville,16700.0,15400.0,Tue Jan 20 2015 01:30:00 GMT-0800 (PST),1
113164,2012,Toyota,Prius c,Two,hatchback,automatic,jtdkdtb34c1509187,pa,fair,54808.0,black,gray,ken pollock nissan llc,11900.0,9000.0,Fri Jan 16 2015 01:00:00 GMT-0800 (PST),0
75538,2011,GMC,Yukon,SLT,suv,automatic,1gks1ce05br150916,ca,poor,61831.0,black,black,rvr,23500.0,24500.0,Wed Dec 31 2014 12:30:00 GMT-0800 (PST),1


In [148]:
print(df_train['sellingprice'].unique())
print(df_train['odometer'].unique())
print(df_train['mmr'].unique())
condition_unique = df_train['sellingprice'].nunique()
odometer_unique = df_train['odometer'].nunique()
mmr_unique = df_train['mmr'].nunique()

print(f"Number of unique values in 'sellingprice': {condition_unique}")
print(f"Number of unique values in 'odometer': {odometer_unique}")
print(f"Number of unique values in 'mmr': {mmr_unique}")


[ 3600. 10400. 37800. ... 57700. 19380. 59900.]
[101930.  42210.   7176. ...  80082.  86425. 107224.]
[  4725.  10350.  41100. ... 109000. 155000. 164000.]
Number of unique values in 'sellingprice': 1642
Number of unique values in 'odometer': 128756
Number of unique values in 'mmr': 1096


We will use the following 13 features:
[   'make_int',
    'model_int',
    'body_int',
    'transmission_int',
    'color_int',
    'interior_int',
    'state_int',
    'year_int',
    'trim_int',
    'condition_int',
    'odometer',
    'mmr',
    'sellingprice']


KeyError: "['make_int', 'model_int', 'body_int', 'transmission_int', 'color_int', 'interior_int', 'state_int', 'year_int', 'trim_int', 'condition_int'] not in index"

In [149]:
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.layers import Embedding, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [152]:
from sklearn import metrics as mt
import tensorflow as tf
from tensorflow import keras
import sys
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'
print(tf.__version__)
print(sys.version)

2.16.1
3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [153]:
# create a tensorflow dataset, for ease of use later
batch_size = 64

def create_dataset_from_dataframe(df_input):

    df = df_input.copy()
    labels = df['price_interval']

    df = {key: value.values[:,np.newaxis] for key, value in df_input[categorical_headers+numeric_headers].items()}

    # create the Dataset here
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    
    # now enable batching and prefetching
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    
    return ds

ds_train = create_dataset_from_dataframe(df_train)
ds_test = create_dataset_from_dataframe(df_test)

### Modeling (5 points total)
- [2 points] Create at least three combined wide and deep networks to classify your data using Keras (this total of "three" includes the model you will train in the next step of the rubric). Visualize the performance of the network on the training data and validation data in the same plot versus the training iterations.
Note: you can use the "history" return parameter that is part of Keras "fit" function to easily access this data.

- [2 points] Investigate generalization performance by altering the number of layers in the deep branch of the network. Try at least two models (this "two" includes the wide and deep model trained from the previous step). Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab to answer: What model with what number of layers performs superiorly? Use proper statistical methods to compare the performance of different models.

- [1 points] Compare the performance of your best wide and deep network to a standard multi-layer perceptron (MLP). Alternatively, you can compare to a network without the wide branch (i.e., just the deep network). For classification tasks, compare using the receiver operating characteristic and area under the curve. For regression tasks, use Bland-Altman plots and residual variance calculations.  Use proper statistical methods to compare the performance of different models.  


### Exceptional Work (1 points total)
5000 students: You have free reign to provide additional analyses.
One idea (required for 7000 level students): Capture the embedding weights from the deep network and (if needed) perform dimensionality reduction on the output of these embedding layers (only if needed). That is, pass the observations into the network, save the embedded weights (called embeddings), and then perform  dimensionality reduction in order to visualize results. Visualize and explain any clusters in the data.